# Процессы

In [1]:
import random
import os
import time
import threading
import multiprocessing
import os

In [2]:
# Типы задач
def cpu_bound_task(n):
    def fib(n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        return fib(n-1) + fib(n-2)

    print(f'{n} -> {fib(n)}')


def io_bound_task(ind):
    print(f'start task {ind}')
    time.sleep(3)
    print(f'finish task {ind}\n', end='')

In [3]:
%%time

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    cpu_bound_task(number)

36 -> 14930352
35 -> 9227465
34 -> 5702887
33 -> 3524578
32 -> 2178309
31 -> 1346269
30 -> 832040
25 -> 75025
20 -> 6765
15 -> 610
10 -> 55
Wall time: 15.8 s


In [4]:
%%time

# CPU bound задача на потоках
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = threading.Thread(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()
    
#время больше, потому что нужно переключаться между потоками

20 -> 6765
15 -> 610
10 -> 55
25 -> 75025
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
Wall time: 16.4 s


In [5]:
%%time

# CPU bound задача на процессах
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = multiprocessing.Process(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()


Wall time: 312 ms


In [6]:
%%time

for ind in range(5):
    io_bound_task(ind)

start task 0
finish task 0
start task 1
finish task 1
start task 2
finish task 2
start task 3
finish task 3
start task 4
finish task 4
Wall time: 15 s


In [7]:
%%time

# IO bound задача на потоках
tasks = []

for ind in range(5):
    task = threading.Thread(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0
start task 1
start task 2
start task 3
start task 4
finish task 0
finish task 1
finish task 2
finish task 3
finish task 4
Wall time: 3.01 s


In [8]:
%%time

# IO bound задача на процессах
tasks = []

for ind in range(5):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

Wall time: 177 ms


In [9]:
# Получение идентификатора процесса
def io_bound_task(ind):
    print(f'start task {ind} with {os.getpid()}')
    time.sleep(3)
    print(f'finish task {ind}')
    
tasks = []

for ind in range(10):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

In [ ]:
# Пулл процессов
from multiprocessing import Pool
# распределение процессов в пулле

def getpid(n):
    time.sleep(2)
    return os.getpid() #получить id процесса

with Pool(3) as p:
    print(p.map(getpid, range(5)))

In [ ]:
# Шаринг ресурсов
share_memory = {
    'count': 0,
}
def share_memory_task():
    print(f'read {share_memory["count"]}')
    share_memory['count'] += 1
    print(f'write {share_memory["count"]}')

tasks = []

for _ in range(5):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

In [ ]:
# Шаринг ресурсов через файл
filename = 'share_memory.tmp'

with open(filename, 'w') as fd:
    fd.write('0')

def share_memory_task():
    with open(filename) as fd:
        count = int(fd.read())
    print(f'read {count}')

    with open(filename, 'w') as fd:
        fd.write(str(count + 1))
    print(f'write {count + 1}')


tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

with open(filename) as fd:
    print(f'COUNT: {fd.read()}')

In [ ]:
from multiprocessing import Manager, Queue


PROCESSES_COUNT = 5
to_process_list = [
    [1, 2, 3], [4, 5, 6], [7, 8, 9],
    [10, 11, 12], [13, 14, 15],
    [16, 17, 18], [19, 20, 21],
    [22, 23, 24], [25, 26, 27]
]


def func(x):
    time.sleep(3)
    return 1 << x #побитовый сдвиг, степени двойки


def producer_func(to_process, queue):
    for elem in to_process:
        res = func(elem)
        queue.put(res)
        print(f"Hey, I've just put {res} to queue!")


def consumer_func(queue):
    while True:
        res = queue.get()
        if res == 'kill':
            break
        print(f"Wow, I found {res} in queue!")


def do_work(to_process_list):
    manager = Manager()
    queue = manager.Queue()
    pool = Pool(PROCESSES_COUNT)

    pool.apply_async(consumer_func, (queue, ))

    jobs = []
    for to_process in to_process_list:
        job = pool.apply_async(producer_func, (to_process, queue))
        jobs.append(job)

    for job in jobs:
        job.get()

    queue.put('kill')
    pool.close()
    pool.join()
    

do_work(to_process_list)